# City Ensemble

In [1]:
import numpy as np
import pandas as pd
from quantile_forest import RandomForestQuantileRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from collections import defaultdict

random_state = 46
np.random.seed(random_state)

# Competition variables.
base_path = "../prediction-interval-competition-ii-house-price/"
alpha = 0.1 

# init dataFrame
df = pd.read_csv(base_path + "dataset.csv", index_col="id", parse_dates=["sale_date"])
df_test = pd.read_csv(base_path + "test.csv", index_col="id", parse_dates=["sale_date"])

## Function Definition

In [2]:
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    """Compute the Winkler Interval Score for prediction intervals.

    Args:
        y_true (array-like): True observed values.
        lower (array-like): Lower bounds of prediction intervals.
        upper (array-like): Upper bounds of prediction intervals.
        alpha (float): Significance level (e.g., 0.1 for 90% intervals).
        return_coverage (bool): If True, also return empirical coverage.

    Returns:
        score (float): Mean Winkler Score.
        coverage (float, optional): Proportion of true values within intervals.
    """
    y_true = np.asarray(y_true)
    lower = np.asarray(lower)
    upper = np.asarray(upper)

    width = upper - lower
    penalty_lower = 2 / alpha * (lower - y_true)
    penalty_upper = 2 / alpha * (y_true - upper)

    score = width.copy()
    score += np.where(y_true < lower, penalty_lower, 0)
    score += np.where(y_true > upper, penalty_upper, 0)

    if return_coverage:
        inside = (y_true >= lower) & (y_true <= upper)
        coverage = np.mean(inside)
        return np.mean(score), coverage

    return np.mean(score)

In [3]:
def feat_eng(df):
    """這個函數處理特徵工程
    
    """
    df['total_val'] = df['land_val'] + df['imp_val']
    df['land_ratio'] = df['land_val'] / df['total_val']

    df['total_baths'] = (
        df['bath_full'] +
        0.75 * df['bath_3qtr'] +
        0.5 * df['bath_half']
    )
    df['is_master_suite'] = (df['bath_full'] > df['beds'] * 0.8).astype(int)
    df['has_garage'] = (df['gara_sqft'] > 0).astype(int)
    df['garage_spots'] = (df['gara_sqft'] / 200).round().astype(int).fillna(0)

    # 把時間拆分成 year/month
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    df['age'] = df['sale_year'] - df['year_built']
    df['living_area'] = df['sqft'] + df['sqft_fbsmt']

    # 這裡是一些類別資料
    for col in ['sale_warning', 'join_status', 'city', 'zoning', 'subdivision', 'submarket']:
        df[col] = df[col].astype(str)
        df[col] = LabelEncoder().fit_transform(df[col])
    
    return df.drop(columns=['sale_date'])

In [4]:
def train_model(train_data):
    """這個函數一次會訓練兩個模型（upper/lower）
    
    """
    lgb_lower = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.05,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        device="gpu"
    )
    lgb_upper = lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.95,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        device="gpu"
    )

    # Split features and target
    x_train = train_data.drop("sale_price", axis=1)
    y_train = train_data["sale_price"]

    lgb_lower.fit(x_train, y_train)
    lgb_upper.fit(x_train, y_train)

    return lgb_lower, lgb_upper

In [5]:
def create_city_list(df):
    city_data_dict = defaultdict(list)

    for idx, row in df.iterrows():
        city = row['city']
        city_data_dict[city].append(row)

    return city_data_dict

## Data Process

In [6]:
# 做特徵工程
df_modify = feat_eng(df)

# Split train/val and test
train_data, val_data = train_test_split(
    df_modify, test_size=0.3, random_state=random_state
)
X_test = df_test.copy()
test_data = feat_eng(X_test)

# 這裡建立一個 city 字典，key 是城市代號，value 是所有該 city 的資料 list（一個物件一筆）
city_df_list_train = create_city_list(train_data)
city_df_list_val = create_city_list(val_data)
city_df_list_test = create_city_list(test_data)

## Train Model

In [9]:
from tqdm import tqdm

val_preds_lower = []
val_preds_upper = []

test_preds_lower = []
test_preds_upper = []

# 每個城市訓練一個模型
for city in tqdm(city_df_list_train.keys(), desc="Processing cities"):
    train_list = city_df_list_train[city]
    val_list = city_df_list_val[city]
    test_list = city_df_list_test[city]

    train_data = pd.DataFrame(city_df_list_train[city])
    val_data = pd.DataFrame(city_df_list_val[city]).drop(columns=['sale_price'])
    test_data = pd.DataFrame(city_df_list_test[city])

    lower_model, upper_model = train_model(train_data)

    val_index = val_data.index
    preds_lower = pd.Series(lower_model.predict(val_data), index=val_index)
    preds_upper = pd.Series(upper_model.predict(val_data), index=val_index)

    test_index = test_data.index
    preds_lower_test = pd.Series(lower_model.predict(test_data), index=test_index)
    preds_upper_test = pd.Series(upper_model.predict(test_data), index=test_index)

    val_preds_lower.append(preds_lower)
    val_preds_upper.append(preds_upper)

    test_preds_lower.append(preds_lower_test)
    test_preds_upper.append(preds_upper_test)

val_preds_lower_all = pd.concat(val_preds_lower).sort_index()
val_preds_upper_all = pd.concat(val_preds_upper).sort_index()

test_preds_lower_all = pd.concat(test_preds_lower).sort_index()
test_preds_upper_all = pd.concat(test_preds_upper).sort_index()

Processing cities:   0%|          | 0/41 [00:00<?, ?it/s]

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3984
[LightGBM] [Info] Number of data points in the train set: 41438, number of used features: 51
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.26 MB) transferred to GPU in 0.028497 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 210000.000000
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3984
[LightGBM] [Info] Number of data points in the train set: 41438, number of used features: 51
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightG

Processing cities:   2%|▏         | 1/41 [02:13<1:29:05, 133.64s/it]

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3683
[LightGBM] [Info] Number of data points in the train set: 6021, number of used features: 48
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (0.18 MB) transferred to GPU in 0.084785 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 229000.000000
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3683
[LightGBM] [Info] Number of data points in the train set: 6021, number of used features: 48
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM

Processing cities:   5%|▍         | 2/41 [04:25<1:26:17, 132.75s/it]

...

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 34, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 424300.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no mo

Processing cities:  98%|█████████▊| 40/41 [1:07:07<00:17, 17.92s/it]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 23, number of used features: 0
[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 408471.406250
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no mo

Processing cities: 100%|██████████| 41/41 [1:07:08<00:00, 98.25s/it]

[LightGBM] [Info] Using GPU Device: Tesla V100-PCIE-32GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 2629899.750000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopp

## Evaluation

In [14]:
y_val = pd.DataFrame(city_df_list_val[city])["sale_price"].sort_index()

lgb_score, coverage = winkler_score(
    y_val, 
    val_preds_lower_all, 
    val_preds_upper_all,
    alpha=alpha,
    return_coverage=True,)
print("The results of lightgbm is this ")

print("Mean Winkler Score:", round(lgb_score, 2))
print("Coverage:", round(coverage * 100, 1), "%")

KeyError: 'sale_price'

## Submission

In [12]:
sample_submission = pd.read_csv(base_path + "sample_submission.csv")
sample_submission["pi_lower"] = test_preds_lower_all
sample_submission["pi_upper"] = test_preds_upper_all
sample_submission.to_csv("submission.csv", index=False)

sample_submission

,id,pi_lower,pi_upper
0,200000,NaN,NaN
1,200001,NaN,NaN
2,200002,NaN,NaN
3,200003,NaN,NaN
4,200004,NaN,NaN
...,...,...,...
199995,399995,NaN,NaN
199996,399996,NaN,NaN
199997,399997,NaN,NaN
199998,399998,NaN,NaN
